# (실습-10) RAG 실습

##실습 개요
1) 실습 목적 <br>
  이번 실습에서는 ColBERT와 LLM을 이용하여 RAG를 구현해 본다. <br>
  RAG 구현을 위해 LLM 프롬프트를 어떻게 사용할 수 있는지 확인한다. <br>
2) 수강 목표
  * RAG 구현을 위한 내부 모듈 및 시스템 파이프라인을 이해한다.
  * ColBERT와 OpenAI API를 사용하여 RAG를 실제로 구현할 수 있다.
  * Function calling을 포함한 프롬프트 엔지니어링 방법에 대해 이해한다.

### 실습 목차
* 1. RAG 개념 이해
* 2. 검색엔진 준비 - ColBERT
* 3. 대화형 IR

### 데이터셋 개요
* 데이터셋: LoTTE 벤치마크의 dev 세트
* 데이터셋 개요
  * LoTTE: ColBERT에서 소개하는 공개 데이터 셋이다.
* 데이터셋 라이센스
  * LoTTE: MIT
* LoTTE 데이터셋 상세 설명 <br>
 LoTTE 데이터 세트는 Wikipedia와 같은 엔티티 중심의 지식 기반에서는 잘 다루지 않을 수 있는 롱테일 주제에 대한 IR 시스템을 평가하기 위해 설계되었습니다. 이번 실습에서는 주로 반려 동물을 다루고 있는 문서를 사용합니다.

### 환경 설정
OpenAI 패키지를 설치한다. <br>
RAG용 검색엔진으로 사용할 ColBERT 패키지를 설치한다.

In [19]:
API_KEY = ""

In [78]:
# OpenAI Python 패키지 설치
# !pip install openai

# 검색엔진을 위해 ColBERT 사용
# !pip install "colbert-ir[faiss-gpu, torch]"

In [1]:
from datasets import load_dataset

# LoTTE dataset 사용

dataset = 'lifestyle'
datasplit = 'dev'

collection_dataset = load_dataset("colbertv2/lotte_passages", dataset)
collection = [x['text'] for x in collection_dataset[datasplit + '_collection']]

queries_dataset = load_dataset("colbertv2/lotte", dataset)
queries = [x['query'] for x in queries_dataset['search_' + datasplit]]

print(f'Loaded {len(queries)} queries and {len(collection):,} passages')
print(queries[0])
print(collection[0])

/home/pervinco/miniconda3/envs/IR/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Loaded 417 queries and 268,893 passages
how much should i feed my 1 year old english mastiff?
In my experience rabbits are very easy to housebreak. They like to pee and poop in the same place every time, so in most cases all you have to do is put a little bit of their waste in the litter box and they will happily use the litter box. It is very important that if they go somewhere else, miss the edge or kick waste out of the box that you clean it up well and immediately as otherwise those spots will become existing places to pee and poop. When you clean the box, save a little bit of waste and put it in the cleaned box so it smells right to them. For a more foolproof method, you can get a piece of wood soaked with their urine and put that in the box along with droppings or cage them so that they are only in their litter box for a week. Generally, if I try the first method and find that they are not using only the box on the first day, I go for the litter box only for a week method. The wo

## 1. RAG 개념 이해

레퍼런스 정보가 있다고 가정하고 그 정보를 토대로 질문에 답하는 예시

In [2]:
# OpenAI 라이브러리 및 필요한 모듈을 가져옵니다.
import os
import json
from openai import OpenAI

# OpenAI API 키를 환경변수에서 설정합니다.
os.environ["OPENAI_API_KEY"] = API_KEY

client = OpenAI()
# 사용할 모델을 설정합니다. 여기서는 gpt-3.5-turbo-1106 모델을 사용합니다.
#llm_model = "gpt-4-1106-preview"
llm_model = "gpt-3.5-turbo-1106"

In [3]:
messages = [
{"role": "user", "content": "너는 누구니"},
{"role": "assistant", "content": "저는 식당 검색 도우미 Good Place 라고 합니다."},
{"role": "user", "content": "강남역 근처 태국음식 추천해줘"}
]

reference = [
{"식당명": "파파야리프", "대표 메뉴": "똠양꿍", "분위기": "이국적/이색적", "기타": "데이트에 적합"},
{"식당명": "할랄가이즈", "대표 메뉴": "할랄", "분위기": "무슬림, 이국적", "기타": "가성비"},
{"식당명": "인더비엣", "대표 메뉴": "팟타이, 쌀국수", "분위기": "깔끔", "기타": "해장"}
]

persona = """
## Name: Good Place

## Role: 식당 검색 도우미

## Instruction
- 사용자의 이전 메시지 정보 및 주어진 검색 결과(JSON 형태로 제공) 정보를 활용하여 간결하게 답변을 생성한다.
- 주어진 검색 결과 정보로 대답할 수 없는 경우는 정보가 부족해서 답을 할 수 없다고 대답한다.
- 사용자가 사용한 언어로 답변을 생성한다.
"""

In [4]:
# 레퍼런스 없이 답변 생성하는 경우 - hallucination 발생

result = client.chat.completions.create(
    model=llm_model,
    messages=messages,
    temperature=0,
    seed=1
)

print(result.choices[0].message)

ChatCompletionMessage(content='강남역 근처에서 태국음식을 즐길 수 있는 몇 가지 좋은 식당을 추천해 드릴게요. \n\n1. 태국집 쏭카오 (Songkran) - 강남구 역삼동 619-14\n2. 태국집 쏭타이 (Song Thai) - 강남구 역삼동 619-14\n3. 태국음식 전문점 쏭쏭 (Song Song) - 강남구 역삼동 619-14\n\n이 식당들은 강남역 근처에 위치해 있으며, 정통 태국 음식을 맛볼 수 있는 곳으로 손님들에게 좋은 평가를 받고 있어요. 방문해보시면 만족하실 거예요!', refusal=None, role='assistant', function_call=None, tool_calls=None)


In [5]:
# 레퍼런스를 토대로 답변 생성하게 하는 경우

content = {
    "이전 메시지 정보": messages,
    "검색 결과": reference
}

msg = [{"role": "system", "content": persona}, {"role": "assistant",  "content": json.dumps(content, ensure_ascii=False, indent=4)}]

result = client.chat.completions.create(
    model=llm_model,
    messages=msg,
    temperature=0,
    seed=1
)

print(result.choices[0].message)

ChatCompletionMessage(content='강남역 근처 태국음식 추천해드릴게요. 파파야리프는 똠양꿍으로 유명하고 데이트에 적합한 이국적인 분위기를 가지고 있어요. 또한 할랄가이즈는 할랄 음식으로 유명하고 가성비가 좋아요. 인더비엣은 깔끔한 분위기와 해장 메뉴로 유명해요. 원하시는 분위기나 메뉴에 맞게 선택해보세요!', refusal=None, role='assistant', function_call=None, tool_calls=None)


## 2. 검색엔진 준비 - ColBERT




빠른 실습을 위해 처음 10,000개의 구절에 대해서만 색인

In [7]:
# 필요한 라이브러리 import

from colbert import Indexer, Searcher
from colbert.infra import Run, RunConfig, ColBERTConfig
from colbert.data import Queries, Collection
import colbert

In [8]:
# 테스트를 위해 전체 문서 중 10000개만 색인 (약 5분 정도 소요됨)

checkpoint = 'colbert-ir/colbertv2.0'
index_name = "test"
doc_maxlen = 300
max_id = 10000

with Run().context(RunConfig(nranks=1, experiment='notebook')):  # nranks specifies the number of GPUs to use
    config = ColBERTConfig(doc_maxlen=doc_maxlen, nbits=2, kmeans_niters=4) # kmeans_niters specifies the number of iterations of k-means clustering; 4 is a good and fast default.                                                                           # Consider larger numbers for small datasets.

    indexer = Indexer(checkpoint=checkpoint, config=config)
    indexer.index(name=index_name, collection=collection[:max_id], overwrite=True)




[Oct 01, 16:30:14] #> Creating directory /home/pervinco/Upstage_Ai_Lab/15-InformationRetrieval.ipynb/experiments/notebook/indexes/test 


#> Starting...
nranks = 1 	 num_gpus = 1 	 device=0
{
    "query_token_id": "[unused0]",
    "doc_token_id": "[unused1]",
    "query_token": "[Q]",
    "doc_token": "[D]",
    "ncells": null,
    "centroid_score_threshold": null,
    "ndocs": null,
    "load_index_with_mmap": false,
    "index_path": null,
    "index_bsize": 64,
    "nbits": 2,
    "kmeans_niters": 4,
    "resume": false,
    "pool_factor": 1,
    "clustering_mode": "hierarchical",
    "protected_tokens": 0,
    "similarity": "cosine",
    "bsize": 64,
    "accumsteps": 1,
    "lr": 1e-5,
    "maxsteps": 400000,
    "save_every": null,
    "warmup": 20000,
    "warmup_bert": null,
    "relu": false,
    "nway": 64,
    "use_ib_negatives": true,
    "reranker": false,
    "distillation_alpha": 1.0,
    "ignore_scores": false,
    "model_name": null,
    "query_maxlen": 32,
    "atte

/home/pervinco/miniconda3/envs/IR/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


[Oct 01, 16:30:18] [0] 		 # of sampled PIDs = 10000 	 sampled_pids[:3] = [6825, 166, 4892]
[Oct 01, 16:30:18] [0] 		 #> Encoding 10000 passages..
[Oct 01, 16:30:30] [0] 		 avg_doclen_est = 180.2436981201172 	 len(local_sample) = 10,000
[Oct 01, 16:30:30] [0] 		 Creating 16,384 partitions.
[Oct 01, 16:30:30] [0] 		 *Estimated* 1,802,436 embeddings.
[Oct 01, 16:30:30] [0] 		 #> Saving the indexing plan to /home/pervinco/Upstage_Ai_Lab/15-InformationRetrieval.ipynb/experiments/notebook/indexes/test/plan.json ..
Clustering 1752437 points in 128D to 16384 clusters, redo 1 times, 4 iterations
  Preprocessing in 0.05 s
  Iteration 3 (1.15 s, search 1.02 s): objective=440053 imbalance=1.324 nsplit=0       
[Oct 01, 16:30:32] Loading decompress_residuals_cpp extension (set COLBERT_LOAD_TORCH_EXTENSION_VERBOSE=True for more info)...
[Oct 01, 16:30:56] Loading packbits_cpp extension (set COLBERT_LOAD_TORCH_EXTENSION_VERBOSE=True for more info)...
[0.036, 0.037, 0.036, 0.034, 0.034, 0.036, 0.036, 

0it [00:00, ?it/s]

[Oct 01, 16:31:33] [0] 		 #> Saving chunk 0: 	 10,000 passages and 1,802,437 embeddings. From #0 onward.


1it [00:11, 11.95s/it]
100%|██████████| 16384/16384 [00:00<00:00, 224743.85it/s]


[Oct 01, 16:31:33] [0] 		 #> Checking all files were saved...
[Oct 01, 16:31:33] [0] 		 Found all files!
[Oct 01, 16:31:33] [0] 		 #> Building IVF...
[Oct 01, 16:31:33] [0] 		 #> Loading codes...
[Oct 01, 16:31:33] [0] 		 Sorting codes...
[Oct 01, 16:31:33] [0] 		 Getting unique codes...
[Oct 01, 16:31:33] #> Optimizing IVF to store map from centroids to list of pids..
[Oct 01, 16:31:33] #> Building the emb2pid mapping..
[Oct 01, 16:31:33] len(emb2pid) = 1802437
[Oct 01, 16:31:33] #> Saved optimized IVF to /home/pervinco/Upstage_Ai_Lab/15-InformationRetrieval.ipynb/experiments/notebook/indexes/test/ivf.pid.pt
[Oct 01, 16:31:33] [0] 		 #> Saving the indexing metadata to /home/pervinco/Upstage_Ai_Lab/15-InformationRetrieval.ipynb/experiments/notebook/indexes/test/metadata.json ..
#> Joined...


In [9]:
# 검색을 위한 searcher 생성
with Run().context(RunConfig(experiment='notebook')):
    searcher = Searcher(index=index_name, collection=collection)


/home/pervinco/miniconda3/envs/IR/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


[Oct 01, 16:31:43] #> Loading codec...
[Oct 01, 16:31:43] Loading decompress_residuals_cpp extension (set COLBERT_LOAD_TORCH_EXTENSION_VERBOSE=True for more info)...
[Oct 01, 16:31:43] Loading packbits_cpp extension (set COLBERT_LOAD_TORCH_EXTENSION_VERBOSE=True for more info)...
[Oct 01, 16:31:44] #> Loading IVF...
[Oct 01, 16:31:44] #> Loading doclens...


100%|██████████| 1/1 [00:00<00:00, 2242.94it/s]

[Oct 01, 16:31:44] #> Loading codes and residuals...



100%|██████████| 1/1 [00:00<00:00, 37.86it/s]


In [10]:
# 문서 검색 - 영어 문서이기 때문에 영어로 질의 생성
queries = ["What are key factors to consider for minimizing stress when moving an aquarium?"]

for query in queries:
  print(f"#> {query}")

  # Find the top-5 passages for this query
  results = searcher.search(query, k=5)

  # Print out the top-k retrieved passages
  for passage_id, passage_rank, passage_score in zip(*results):
      print(f"\t [{passage_rank}] \t\t {passage_score:.1f} \t\t {searcher.collection[passage_id]}")

#> What are key factors to consider for minimizing stress when moving an aquarium?

#> QueryTokenizer.tensorize(batch_text[0], batch_background[0], bsize) ==
#> Input: What are key factors to consider for minimizing stress when moving an aquarium?, 		 True, 		 None
#> Output IDs: torch.Size([32]), tensor([  101,     1,  2054,  2024,  3145,  5876,  2000,  5136,  2005,  7163,
         4328,  6774,  6911,  2043,  3048,  2019, 18257,  1029,   102,   103,
          103,   103,   103,   103,   103,   103,   103,   103,   103,   103,
          103,   103], device='cuda:0')
#> Output Mask: torch.Size([32]), tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0')

	 [1] 		 21.1 		 When moving an aquarium (from an old location close to the new one) consider the following: Moving the aquarium means stress for inhabitants and you. So, dont place the aquarium at a worse location than before ((more) direct impact of sunlight w

## 3. 대화형 IR

준비된 검색엔진과 LLM을 활용하셔 대화형 IR 시스템 구현

In [25]:
# OpenAI 라이브러리 및 필요한 모듈을 가져옵니다.
import os
import json
from openai import OpenAI

# OpenAI API 키를 환경변수에서 설정합니다.
os.environ["OPENAI_API_KEY"] = API_KEY

client = OpenAI()
# 사용할 모델을 설정합니다. 여기서는 gpt-3.5-turbo-1106 모델을 사용합니다.
#llm_model = "gpt-4-1106-preview"
llm_model = "gpt-4o"

In [26]:

persona_qa = """
## Role: 반려 동물 전문가

## Instructions
- 사용자의 이전 메시지 정보 및 주어진 Reference(JSON 형태로 제공) 정보를 활용하여 간결하게 답변을 생성한다.
- 주어진 검색 결과 정보로 대답할 수 없는 경우는 다른 대답 하지 말고 "정보가 부족해서 답을 할 수 없습니다"라고 대답한다.
- 한국어로 답변을 생성한다.
"""

persona_function_calling = """
## Role: 반려 동물 전문가

## Instruction
- 사용자가 대화를 통해 반려 동물 관련된 지식이나 도움을 요청하면 search api를 호출할 수 있어야 한다.
- Search api에 필요한 standalone_query는 영어로 생성한다.
- 반려 동물과 관련되지 않은 나머지 대화 메시지에는 적절한 대답을 생성한다.
"""

tools = [
    {
        "type": "function",
        "function": {
            "name": "search",
            "description": "search knowledge from the user messages sent and received.",
            "parameters": {
                "properties": {
                    "standalone_query": {
                        "type": "string",
                        "description": "English query suitable for use in search from the user messages history."
                    }
                },
                "required": ["standalone_query"],
                "type": "object"
            }
        }
    },
]


In [27]:
# 대화형 Agent를 위한 dialog manager 구현

import json

def conversation_search(messages, persona, tools):
    msg = [{"role": "system", "content": persona}] + messages

    ## 질의 및 히스토리를 모델에 입력하고 결과를 얻는다.
    result = client.chat.completions.create(
        model=llm_model,
        messages=msg,
        tools=tools,
        temperature=0,
        seed=1
    )
    response_message = result.choices[0].message

    ## 만약 질의가 주제(애완동물 관련)를 벗어난 경우 function call이 호출되지 않고, 주제에 적합한 경우 function call이 유도된다.
    if response_message.tool_calls:
        for tool_call in response_message.tool_calls:
            function_args = json.loads(tool_call.function.arguments)
            standalone_query = function_args.get("standalone_query")

            results = searcher.search(standalone_query, k=3)
            print("검색어>>: ", standalone_query)
            print()
            print("검색결과>>: ")
            
            retrieved_context = []
            for passage_id, passage_rank, passage_score in zip(*results):
                retrieved_context.append(searcher.collection[passage_id])
                print(f"\t [{passage_rank}] \t\t {passage_score:.1f} \t\t {searcher.collection[passage_id]}")

            msg = [{"role": "system", "content": persona_qa}] + messages
            msg.append({"role": "user", "content": json.dumps(retrieved_context)})
            qaresult = client.chat.completions.create(
                    model=llm_model,
                    messages=msg,
                    temperature=0,
                    seed=1
                )
            print("LLM 대답>>:")
            print(qaresult.choices[0].message.content)
    else:
        print("LLM 대답>>:")
        print(response_message.content)

In [28]:

messages = [
{"role": "user", "content": "안녕하세요, 저는 새로운 개 주인이고 방금 영국 마스티프를 입양했어요. 그를 돌보는 데 대해 몇 가지 팁을 줄 수 있나요?"},
{"role": "assistant", "content": "물론이죠! 영국 마스티프 같은 대형견을 돌보는 것은 매우 보람찬 일입니다. 그들은 균형 잡힌 식단, 규칙적인 운동, 그리고 정기적인 수의사 검진이 필요합니다. 그들의 돌봄에 대해 구체적으로 궁금한 점이 있나요?"},
{"role": "user", "content": "네, 특히 그의 식단에 대해 걱정이 되네요. 1살 영국 마스티프에게 얼마나 많은 양을 먹여야 하나요?"},
]

conversation_search(messages, persona_function_calling, tools)


검색어>>:  How much food should a 1-year-old English Mastiff eat?

검색결과>>: 
	 [1] 		 20.0 		 I have a 2 1/2 year old bull mastiff. I have been feeding him Blue Buffalo since I got him at 8 weeks old. He is very lean and active for a bull mastiff. I feed him about 3-4 cups twice a day which averages about 130.00 a month. It is very important that you can afford this breed. I just had to take mine to the vet because he developed some sort of allergies on his skin, eyes and ears and the vet bill was $210.00 with all his medication. This wasnt an option I had to take him an get all his meds or he would have gotten worse. Theyre just like your children, you can expect things to come up and you need to be able to care for them.
	 [2] 		 19.1 		 I breed mastiffs so I will try to help you with this: Age Amount 4-8 weeks 3-4 cups per day spread between 3-4 meals 8-12 weeks 4-6 cups per day spread between 3-4 meals 12-16 weeks 6-8 cups per day spread between 3-4 meals 4 to 6 months 8-10 cups per da

In [29]:

messages = [
{"role": "user", "content": "저는 고양이를 입양하고 싶은데, 최근에 청각장애가 있는 고양이를 만났어요. 조금 걱정이 되네요. 특별히 고려해야 할 점이 있나요?"},
{"role": "assistant", "content": "청각장애가 있는 고양이를 입양하는 것은 독특하고 보람찬 경험이 될 수 있습니다. 이들은 일반적으로 안전한 실내 환경과 명확한 시각적 의사소통이 필요합니다. 특별히 걱정되는 점이 있나요?"},
{"role": "user", "content": "네, 그들의 행동에 대해 궁금해요. 청각장애가 있는 고양이는 다른 고양이들보다 더 공격적인가요?"}
]

conversation_search(messages, persona_function_calling, tools)


검색어>>:  Are deaf cats more aggressive than hearing cats?

검색결과>>: 
	 [1] 		 25.7 		 We had a deaf cat, sadly passed away only about a month ago, and to be honest we didnt handle her any differently than our other cat. The things to bear in mind are: Theyre easier to surprise and every cat might react a little differently if startled, particularly from sleep. Ours just kind of jumped up, but others might be a bit more aggressive. They wont react to shouts or loud noises. We used to clap our hands to get the attention of our cats when they were doing things like clawing furniture. That would usually get them to stop, but that was totally useless with her once she went deaf. At that point wed just pick her up and remove her from the area, which seemed to work. Deaf cats still like attention, grooming, and petting. I found that she became even more of a lap cat once she became deaf, possibly because of getting touch feedback from us. In any event, cats are pretty adaptable and deafness wil

In [30]:

messages = [
{"role": "user", "content": "낙타를 키우고 싶은데 어떤 준비가 필요한가요?"}
]

conversation_search(messages, persona_function_calling, tools)

검색어>>:  What preparations are needed to keep a camel as a pet?

검색결과>>: 
	 [1] 		 14.8 		 Sugar gliders are very high maintenance pets. They require an enormous amount of attention and care, and they are very fickle little things. Do not get a single glider - they have a huge amount of trouble living alone. Get two or more, or dont get any. Your house will need to be thoroughly prepared for sugar gliders. They can fit in tiny spaces, climb on everything, and glide to the last places youd imagine. They chew wires, pee and poop everywhere, and knock stuff over left and right. Their diet requires live insects and grubs, and youll probably end up with a bunch of renegade cricket escapees living in your house. Their diet must be varied constantly, but any given glider will have its own tastes, and will reject foods it doesnt like. You need to have a constant supply of fruits, vegetables, and insects at all times, and youll probably have to hand feed the gliders a significant portion of the 

In [31]:
# 지식에 대한 질문이 아닌 경우

messages = [
{"role": "user", "content": "저는 고양이를 입양하고 싶은데, 최근에 청각장애가 있는 고양이를 만났어요. 조금 걱정이 되네요. 특별히 고려해야 할 점이 있나요?"},
{"role": "assistant", "content": "청각장애가 있는 고양이를 입양하는 것은 독특하고 보람찬 경험이 될 수 있습니다. 이들은 일반적으로 안전한 실내 환경과 명확한 시각적 의사소통이 필요합니다. 특별히 걱정되는 점이 있나요?"},
{"role": "user", "content": "그런데 너는 누구니?"}
]

#conversation_search(messages, persona_function_calling, tools, {"type": "function", "function": {"name": "searcher"}})
conversation_search(messages, persona_function_calling, tools)

LLM 대답>>:
저는 반려 동물 전문가로서, 반려 동물에 관한 질문이나 도움을 드리기 위해 여기 있습니다. 반려 동물에 대해 궁금한 점이 있으면 언제든지 물어보세요!


#Reference

## Required Package

openai==1.7.2 <br>
colbert-ir==0.2.14 <br>
torch==1.13.1 <br>
faiss-gpu==1.7.0

